In [ ]:
import pickle as pkl
import pandas as pd
import numpy as np
import pdb
import matplotlib.pyplot as plt

In [ ]:
def get_result_settings(result_dict):
    """
    from result dict with [modality][feature_type][metric], return all the key values
    Returns:
       modalities (list), feature_types(list)
    """
    modalities = []
    feature_types = []
    for modality, feature_type_dict in result_dict.items():
        modalities.append(modality)
        for feature_type in feature_type_dict:
            feature_types.append(feature_type)
    return [set(val) for val in [modalities, feature_types]]

def get_unimodal_result_dict(task):
    with open(f'../data/supervised_pipeline/{task}_performance_tracker.pkl', 'rb') as f:
        result_dict = pkl.load(f) # [task][modality][feature_type][metric]
    return result_dict

def get_fusion_result_dict(task):
    """
    actually this one should contain all the modalities
    """
    with open(f'../data/supervised_pipeline/{task}_late_fusion_performance_tracker.pkl', 'rb') as f:
        result_dict = pkl.load(f)
    return result_dict

In [ ]:
BASE_DIR = '../data/'
TASKS = ['los', 'mort']
metric = 'auc'
BAR_WIDTH = 0.2

for task in TASKS:
    result_dict = get_unimodal_result_dict(task)
    modalities, feature_types = get_result_settings(result_dict)
    xticks = modalities
    #pdb.set_trace()
    xtick_pos = np.arange(len(xticks))
    colors = [plt.cm.tab20(j) for j in range(len(feature_types))]
    fig, ax = plt.subplots()
    for j, feature_type in enumerate(feature_types):
        metrics = [result_dict[modality][feature_type][metric] for modality in modalities if feature_type in result_dict[modality]]
        ax.bar(xtick_pos + j * BAR_WIDTH, metrics, width=BAR_WIDTH, label=feature_type)
        [ax.text(xtick_pos[i] + j * BAR_WIDTH, value + 0.01, f'{value:.3f}', ha='center') for i, value in enumerate(metrics)] 
    # Set the xticks and labels
    ax.set_xticks(xtick_pos)
    ax.set_xticklabels(xticks)

    # Set the y-label and the title
    ax.set_ylabel('AUC')
    ax.set_title(f'{task.upper()} Predictions')
    ax.set_ylim(0, 1)

    # Add the legend
    ax.legend(loc='lower left')

    # Show the plot
    plt.show()
        

# let's not get too fancy. just make a table of results

In [ ]:
from collections.abc import Iterable
for task in TASKS:
    result_dict = get_fusion_result_dict(task)
    modalities, feature_types = get_result_settings(result_dict)
    # create empty lists to store the values for each column
    modality_list = []
    setting_list = []
    metrics_dict = {}
    for modality, settings_dict in result_dict.items():
        # loop through the inner keys (settings) for each modality
        for setting, metrics in settings_dict.items():
            # append the values to the corresponding lists
            modality_list.append(modality)
            setting_list.append(setting)
            # update the metrics dictionary with any new metrics
            metrics = {metric: val for metric, val in metrics.items() if not isinstance(val, Iterable)}
            for metric, val in metrics.items():
                if metric not in metrics_dict:
                    metrics_dict[metric] = []
                metrics_dict[metric].append(val)

    # create the DataFrame from the lists and metrics dictionary
    # rename modality list
    renamed_modalities = []
    renamed_settings = []
    for modality in modality_list:
        if modality == 'ecg':
            renamed_modalities.append('ecg feats')
        else:
            renamed_modalities.append(modality.replace('_', ' '))
    for setting in setting_list:
        split_name = setting.split('-')
        topic_name = 'topics(' + split_name[-1] + ')'
        if len(split_name) > 1:
            ecg_name = 'ecg feats + '
        else:
            ecg_name = ''
        renamed_settings.append(ecg_name + topic_name)
    df = pd.DataFrame({
        'modality': renamed_modalities,
        'setting': renamed_settings,
        **metrics_dict
    })

    # print the resulting DataFrame
    print(f'task: {task}')
    print(df.to_string(index=False))
    print(df.to_latex(index=False, float_format="%.3f"))

In [ ]:
# make it latex
print(df.to_latex(index=False))